# OpenStreet Data Wrangling with Python and SQL
# This one single file will include all queries, wrangling and details related to every part of the project, to bring more consistency to our project as we try equally hard with our Map. 

### Project Summary
### Map area: Cleveland, Ohio, United States

This map is of a place that is home to my favourite NBA Team, the Cavaliers. This reveals some inconsistencies in data and contributes to its improvement on OpenStreetMap.org.

Data auditing and processing:

## After downloading and auditing the Ohio area dataset, I noticed some inconsistencies in representing data:
1) Street names were inconsistent.
Abbreviations St -> Street
Dots at the end St. -> Street
Lowercase street -> Street

2) Zip codes had different formats.
5-4 digit formats. 
State abbreviations with zip code OH44118 -> 44118

Changed all street names to consistent forms and zip codes to correct formats.


## Importing the required libraries/packages.

In [30]:
import xml.etree.cElementTree as ET
from collections import defaultdict
from operator import itemgetter
import re
import codecs
import json
import pprint
import maincode #This is the parent file from which we derive our REGEX.
OSMFILE = "cleveland_ohio.osm"
import sqlite3

In [31]:
tags = dict() #To check how many types of tags we have available in our data.
for _,j in ET.iterparse(OSMFILE):
    if j.tag in tags:
        tags[j.tag]+=1
    else:
        tags[j.tag]=1
        
print(tags)

{'bounds': 1, 'tag': 1162747, 'node': 1857212, 'nd': 2164787, 'way': 197433, 'member': 26049, 'relation': 3839, 'osm': 1}


In [3]:
street_type_re = maincode.street_type_re

expected = maincode.expected

# After auditing street names these new mappings were created to map variations to expected
street_name_mapping = {
    'St': 'Street',
    'Ave': 'Avenue',
    'Ave.': 'Avenue',
    'Blvd': 'Boulevard',
    'Blvd.': 'Boulevard',
    'Dr': 'Drive',
    'Dr.': 'Drive',
    'Ln': 'Lane',
    'Pkwy': 'Parkway',
    'Rd': 'Road',
    'Rd.': 'Road',
    'St': 'Street',
    'St.': 'Street',
    'Street.': 'Street',
    'ave': 'Avenue'
}


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


# is this element in the format of expected street key
def is_street_name(elem):
    return elem.attrib['k'] == "addr:street"


# audit method to extract the street types
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

if False:
    pprint.pprint(dict(audit(OSMFILE)))


#Update names to expected values
def update_name(name, mapping):
    better_name = name
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            better_name = re.sub(street_type, mapping[street_type], name)
    return better_name


#REGEX to test keys
lower = maincode.lower
lower_colon = maincode.lower_colon
problemchars = maincode.problemchars

# The valid Cleveland,Ohio zip codes should fall in the range of 43001 and 45999
def update_zip_code(zip):
    matched = zip_code_re.match(zip)
    if matched:
        zip_code = int(matched.group(1))
        if zip_code >= 43001 and zip_code <= 45999:
            return zip_code
    return None

# Helper function to see if the value is a street
def is_street_tag(key_array):
    if len(key_array) > 1 and key_array[0] == 'addr' and key_array[1] == 'street':
        return True
    return False


# Assumes the tag is a street tag with only one : that splits it
def is_street_tag_only(street_tag):
    return len(street_tag) == 2


def is_postal_tag(key_array):
    if len(key_array) > 1 and key_array[1] == 'postcode':
        return True
    return False

tiger_left = re.compile(r'^tiger:zip_left$')
tiger_right = re.compile(r'^tiger:zip_right$')


# for auditing tiger zips used
zips_from_tiger = set()

# search an element for tiger zip tags to add to our nodes
# if it has a postal code, use that...else check tiger tags
# if it has either a left or right only return that cleaned zip code
# if it has a left and right, check they are equal and return
# if not equal, or not a valid zip or does not contain tag, return none
def get_best_matched_zip(element):
    left_zip = None
    right_zip = None
    postal_zip = None
    for tag in element.iter("tag"):
        if 'k' in tag.attrib:
            key_array = tag.attrib['k'].split(":")
            if is_postal_tag(key_array):
                postal_zip = update_zip_code(tag.attrib['v'])
                break
            matched = tiger_left.search(tag.attrib['k'])
            if matched:
                left_zip = update_zip_code(tag.attrib['v'])
                continue
            matched = tiger_right.search(tag.attrib['k'])
            if matched:
                right_zip = update_zip_code(tag.attrib['v'])
                continue
    if postal_zip:
        return postal_zip
    if left_zip and right_zip and left_zip == right_zip:
        zips_from_tiger.add(left_zip)
        return left_zip
    if left_zip:
        zips_from_tiger.add(left_zip)
        return left_zip
    if right_zip:
        zips_from_tiger.add(right_zip)
        return right_zip
    return None

#Filter list so we do not overwrite our primary schema keys
keys_to_ignore = ["type", "id", "visible", "created", "address", "addr:postcode"]
tiger_tag = maincode.tiger_tag
tag_keys = defaultdict()



In [4]:
zip_code_re = re.compile(r'(\d{5})',re.UNICODE) #Rewrite the zip code REGEX

# The following schema code is borrowed from the schema.py Udacity code.

In [5]:

SCHEMA = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


# The next function defines our "SHAPER" that helps frame our data into the required format to be then converted to .csv files.

In [6]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way":
        if element.tag == "node":
            node['type'] = "node"
        else:
            node['type'] = "way"

        node['id'] = element.attrib['id']

        if 'visible' in element.attrib:
            node['visible'] = element.attrib['visible']

        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = []
            node['pos'].append(float(element.attrib['lat']))
            node['pos'].append(float(element.attrib['lon']))

        for c in CREATED:
            if c in element.attrib:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][c] = element.attrib[c]

        if element.tag == "way":
            zip_code = get_best_matched_zip(element)
            # added postal codes to the address when present
            # clean zip and ignore non standard zips before attempting to add
            if zip_code:
                if 'address' not in node:
                    node['address'] = {}
                node['address']['postcode'] = zip_code

            for tag in element.iter("tag"):
                matched = problemchars.search(tag.attrib['k'])
                if matched is None:
                    key_array = tag.attrib['k'].split(':')
                    value = tag.attrib['v']
                    
                    if is_street_tag(key_array):
                        if 'address' not in node:
                            node['address'] = {}
                        # only add the tag if it is just the street, no extra : delimiter
                        if is_street_tag_only(key_array):
                            node['address']['street'] = update_name(value, street_name_mapping)

                    if 'k' in tag.attrib and tag.attrib['k'] not in keys_to_ignore:
                        matched = tiger_tag.search(tag.attrib['k'])
                        if not matched:
                            tag_keys[tag.attrib['k']] = 1
                            node[tag.attrib['k']] = value

            for tag in element.iter("nd"):
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in tag.attrib:
                    node['node_refs'].append(tag.attrib['ref'])

        return node
    return None

In [7]:
for i,j in ET.iterparse(OSMFILE): #Calling the main shaper function on each element.
    j = shape_element(j)

# The next code was referenced from the Quiz number 11 of the final lesson.


In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: v for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSMFILE, validate=True)


# After importing the data from .csv files to the in-memory database using ".mode csv", we will save it to a database in our storage for better access and backup.

## Some basic details before we begin the querying using SQLite3 drivers for Python: 

## 1. Size of the uncompressed .OSM file : 415.6Mb
## 2. Size of the SQL database : 297.2Mb

### The number of nodes and ways: 

In [26]:
conn = sqlite3.connect("shantanu2.db")
cursor = conn.cursor()

#Executing and printing the queries. 
cursor.execute("select count(id) from nodes;")
print('There are {} nodes in our database.'.format(cursor.fetchall()[0][0]))

cursor.execute("select count(id) from ways;")
print('There are {} ways in our database.'.format(cursor.fetchall()[0][0]))

There are 1857212 nodes in our database.
There are 197433 ways in our database.


### The number of unique users:

In [28]:
cursor.execute("select count(distinct(uid)) from (select uid from nodes union select uid from ways);")
print('There are {} unique users in our database.'.format(cursor.fetchall()[0][0]))

There are 1360 unique users in our database.


### The number of nodes in the biggest way and the biggest way:

In [27]:
cursor.execute("select id, count(*) as nodes_count from ways_nodes group by id order by nodes_count desc limit 1;")
way_id, count = cursor.fetchall()[0]
print("There are {} nodes in the biggest way of our database. The Way id is {}.".format(count, way_id))

cursor.execute("select * from ways_tags where id = {};".format(way_id))
print('This way is:')
pprint.pprint(cursor.fetchall())

There are 1754 nodes in the biggest way of our database. The Way id is 37881132.
This way is:
[('37881132', 'name', 'West Branch Rocky River', 'regular'),
 ('37881132', 'source', 'Yahoo;bing', 'regular'),
 ('37881132', 'waterway', 'river', 'regular')]


### The number of bridges in Ohio:

In [32]:
cursor.execute("select count(key) from ways_tags where key = 'bridge' and value != 'yes' group by key;")
print("There are {} bridges in Ohio.".format(cursor.fetchall()[0][0]))

There are 50 bridges in Ohio.


### The top 7 postcodes in the dataset:

In [33]:
cursor.execute("select value, count(*) as count from nodes_tags where key = 'postcode' group by value order by count desc limit 7;")
print("The maximum count of top 7 postcodes in this dataset are:")
pprint.pprint(cursor.fetchall())

The maximum count of top 7 postcodes in this dataset are:
[('44118', 450),
 ('44124', 209),
 ('44113', 142),
 ('44115', 103),
 ('44106', 94),
 ('44123', 85),
 ('44114', 75)]


# CONCLUSION : 

# During my analysis, I've seen large amount of data that has not been correctly formatted and cleaned. But I successfully iterparsed this data and corrected streets, zip codes etc. The bigger issue is that the .osm data has a lot of inconsistencies. Sometimes it's very difficult to find these inconsistencies.

## There is a lot of work to be done to complete this map.

## OpenStreetMap data is not perfect as any human modified project isn't. It'll take a lot of time to find and clean all human-made errors. But we've made our first step. We modified street names and made them more consistent and uniform. Then we transformed XML to CSV format and imported it into an SQL database. And finally we answered some interesting questions using SQL queries.

## Additional ideas:

## In my opinion there's two ways to improve OpenStreenData project.
First of all it's extremely important to attract more people to improving maps. My suggestion would be the use of gamification. It's reasonable to establish ranking systems like Kaggle or badge systems like Khan Academy.

## Benefits of this:
### Increase in productivity
### Help to retain high performers by involving them into moderation
## Anticipated problems:
### Has very small effect on results
### Creates competition that can be counterproductive


## Second is to use different sources to cross-validate inconsistencies and empty spots on OS maps.